<a href="https://colab.research.google.com/github/krsnamurari/NLP_Classificacao/blob/main/Projeto_Final_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**UNIVERSIDADE FEDERAL DE PERNAMBUCO**<br>
**CENTRO DE INFORMÁTICA**<br>
<br>
**CURSO DE PÓS-GRADUAÇÃO EM CIÊNCIA DE DADOS**<br>
**DISCIPLINA: PROJETO EM CIÊNCIA DE DADOS**<br>
**PROFESSOR: LUCIANO BARBOSA**<br>
<br>
**ALUNOS: MICHELE VANESSA SERCUNDES NUNES e<br>
           KRSNA MURARI DE ALBUQUERQUE RODRIGUES**


---

#**Importação de bibliotecas**

In [1]:
!pip install optuna tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from optuna.samplers import TPESampler
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import optuna
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import cross_val_score
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset, random_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

#**1. Coleta e Pré-processamento de Dados para Análise de Tweets**

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


pré-processamento dos dados

In [4]:
# Baixar recursos necessários do NLTK
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

# Carregar o dataset
df = pd.read_csv('/content/drive/MyDrive/mLabel_tweets.csv')

# Função para limpeza dos tweets
def clean_tweet(tweet):
    # Remover menções, hashtags, URLs e caracteres especiais
    tweet = re.sub(r'(@[A-Za-z0-9_]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^RT|http.+?', '', tweet)
    # Remover números
    tweet = re.sub(r'\d+', '', tweet)
    # Converter para minúsculas
    tweet = tweet.lower()
    return tweet

# Limpeza e tokenização dos tweets
df['tweet'] = df['tweet'].apply(clean_tweet)
df['tweet'] = df['tweet'].apply(nltk.word_tokenize)

# Remoção de stop words e lemmatização
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
df['tweet'] = df['tweet'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x if word not in stop_words])

# Juntar os tokens de volta em strings
df['tweet'] = df['tweet'].apply(lambda x: ' '.join(x))

# Removendo a coluna ID
df = df.drop(columns = ["ID"])

# Vetorização dos tweets usando TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['tweet'])

# Codificação de labels
y = pd.get_dummies(df['labels']).values

# Dividir o dataset em conjuntos de treino, validação e teste
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Para outros modelos, continue usando a vetorização TF-IDF
X_tfidf = vectorizer.fit_transform(df['tweet'])
y_tfidf = y  # Já definido anteriormente

# Extraindo os rótulos únicos
unique_labels = set(label for label_list in df['labels'].str.split() for label in label_list)

# Inicializando o MultiLabelBinarizer
mlb = MultiLabelBinarizer(classes=sorted(unique_labels))

# Aplicando a transformação
df['labels'] = df['labels'].str.split()  # Dividindo os rótulos
labels_binarized = mlb.fit_transform(df['labels'])

# Criando um DataFrame para os rótulos binarizados
labels_df = pd.DataFrame(labels_binarized, columns=mlb.classes_)

# Juntando os rótulos binarizados ao DataFrame original
df = df.join(labels_df)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
df.to_csv('df_final.csv', sep = ',', index = False)

#**2. Análise de Dados**

In [6]:
df.describe()

,conspiracy,country,ineffective,ingredients,mandatory,none,pharma,political,religious,rushed,side-effect,unnecessary
count,9921.000000,9921.000000,9921.000000,9921.000000,9921.000000,9921.000000,9921.000000,9921.000000,9921.000000,9921.000000,9921.00000,9921.000000
mean,0.049088,0.020260,0.168531,0.043947,0.078923,0.063401,0.128314,0.063098,0.006451,0.148775,0.38353,0.072775
std,0.216062,0.140896,0.374356,0.204988,0.269633,0.243695,0.334456,0.243152,0.080062,0.355885,0.48627,0.259780
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.00000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000


#**3. Separação dos dados em treinamento, validação e teste**

In [7]:
# Primeiro, dividimos os dados em conjunto de treino e temporário (combinação de validação e teste)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)

# Agora, dividimos os dados temporários em conjuntos de validação e teste
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

#**4. Seleção de 4 algoritmos de predição**

**Seleção de 4 algoritmos de predição:**<br>
**- Naive Bayes**<br>
**- SVC (Support Vector Classifier)**<br>
**- Gaussian Naive Bayes**<br>
**- Random Forest**

#**5. Treinamento com seleção de hiperparâmetros utilizando o OPTUNA**

**Criação das funções objetivos**

In [8]:
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import cross_val_score

# Função de objetivo para Naive Bayes
def objective_nb(trial):
    alpha = trial.suggest_float('alpha', 1e-4, 1e2, log=True)
    clf = OneVsRestClassifier(MultinomialNB(alpha=alpha))
    return cross_val_score(clf, X_train, y_train, n_jobs=-1, cv=3).mean()

# Função de objetivo para SVM
def objective_svm(trial):
    C = trial.suggest_float('C', 1e-4, 1e4, log=True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid'])
    gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])
    clf = OneVsRestClassifier(SVC(C=C, kernel=kernel, gamma=gamma))
    return cross_val_score(clf, X_train, y_train, n_jobs=-1, cv=3).mean()


In [9]:
from sklearn.naive_bayes import GaussianNB

# Função de objetivo para GaussianNB
def objective_Gnb(trial):
    var_smoothing = trial.suggest_float('var_smoothing', 5e-10, 5e-9)
    clf = GaussianNB(var_smoothing=var_smoothing)
    y_train_1d = np.argmax(y_train, axis=1)
    X_train_1d = X_train.toarray()
    return cross_val_score(clf, X_train_1d, y_train_1d, n_jobs=-1, cv=3).mean()

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

# Função de objetivo para Random Forest Classifier
def objective_rf(trial):
  n_estimators = trial.suggest_int('n_estimators',50, 150)
  criterion = trial.suggest_categorical('criterion', ['gini', 'entropy', 'log_loss'])
  class_weight = trial.suggest_categorical('class_weight', ['balanced', 'balanced_subsample', None])
  clf = RandomForestClassifier(n_estimators=n_estimators, criterion=criterion,
                               class_weight=class_weight, max_depth=2, random_state=0)
  return cross_val_score(clf, X_train, y_train, n_jobs=-1, cv=3).mean()

In [11]:
def objective_master(trial):
    model_type = trial.suggest_categorical('model_type', ['naive_bayes', 'svm', 'GaussianNB', 'rf'])

    if model_type == 'naive_bayes':
      return objective_nb(trial)
    elif model_type == 'svm':
      return objective_svm(trial)
    elif model_type == 'GaussianNB':
      return objective_Gnb(trial)
    elif model_type == 'rf':
      return objective_rf(trial)


In [12]:
df.head()

,tweet,labels,conspiracy,country,ineffective,ingredients,mandatory,none,pharma,political,religious,rushed,side-effect,unnecessary
0,astrazeneca made kidney cell little girl abort...,[ingredients],0,0,0,1,0,0,0,0,0,0,0,0
1,begin please find safe alternative vaccine uk ...,[side-effect],0,0,0,0,0,0,0,0,0,0,1,0
2,well mean congratulation covid first ever thin...,[side-effect],0,0,0,0,0,0,0,0,0,0,1,0
3,u wish vaccine given vaccine passport go abroa...,[mandatory],0,0,0,0,1,0,0,0,0,0,0,0
4,trying speak writing letter government speakin...,"[side-effect, rushed]",0,0,0,0,0,0,0,0,0,1,1,0


In [13]:
study = optuna.create_study(direction='maximize', study_name='NLP Classification', sampler=TPESampler(seed=123))
study.optimize(objective_master, n_trials=10)

# Imprimir os resultados
print("Número do Melhor Trial:", study.best_trial.number)
print("Melhores Parâmetros:", study.best_trial.params)
print("Melhores Valores de Métrica:", study.best_trial.values)

[I 2024-01-27 02:29:00,282] A new study created in memory with name: NLP Classification
[I 2024-01-27 02:29:05,159] Trial 0 finished with value: 0.051123502168227586 and parameters: {'model_type': 'naive_bayes', 'alpha': 2.0740241962891877}. Best is trial 0 with value: 0.051123502168227586.
[I 2024-01-27 02:29:41,396] Trial 1 finished with value: 0.0 and parameters: {'model_type': 'svm', 'C': 0.13706928443177713, 'kernel': 'poly', 'gamma': 'auto'}. Best is trial 0 with value: 0.051123502168227586.
[I 2024-01-27 02:30:49,812] Trial 2 finished with value: 0.0 and parameters: {'model_type': 'rf', 'n_estimators': 114, 'criterion': 'gini', 'class_weight': 'balanced'}. Best is trial 0 with value: 0.051123502168227586.
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(
[I 2024-01-27 02:33:13,180] Trial 3 finished with value: 0.3018442846093488 and paramet

Número do Melhor Trial: 3
Melhores Parâmetros: {'model_type': 'GaussianNB', 'var_smoothing': 2.451655277057877e-09}
Melhores Valores de Métrica: [0.3018442846093488]


#**6. Avaliação do melhor modelo selecionado pelo Optuna usando o MLflow e o conjunto de testes**

In [14]:
df = df.drop(columns='labels')
df.columns

Index(['tweet', 'conspiracy', 'country', 'ineffective', 'ingredients',
       'mandatory', 'none', 'pharma', 'political', 'religious', 'rushed',
       'side-effect', 'unnecessary'],
      dtype='object')

In [15]:
!pip install mlflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.6/147.6 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.4 MB/s eta 0:00:00


In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score, hamming_loss
import mlflow
import mlflow.sklearn

def eval_multilabel_metrics(actual, pred):
    accuracy = accuracy_score(actual, pred)
    hamming = hamming_loss(actual, pred)
    return accuracy, hamming

# Carregar os dados
df = pd.read_csv("/content/df_final.csv")
df = df.drop(columns='labels')

X = df['tweet']
y = df[['conspiracy', 'country', 'ineffective', 'ingredients', 'mandatory', 'none', 'pharma', 'political', 'religious', 'rushed', 'side-effect', 'unnecessary']]

# Dividir os dados
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Vetorização
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train).toarray()
X_test_vec = vectorizer.transform(X_test).toarray()

# Definição e treinamento do modelo com MLflow
var_smoothing = 2.451655277057877e-09

with mlflow.start_run():
    naive_bayes_clf = GaussianNB(var_smoothing=var_smoothing)
    model = OneVsRestClassifier(naive_bayes_clf)
    model.fit(X_train_vec, y_train)

    # Predição e avaliação
    predicted_qualities = model.predict(X_test_vec)
    accuracy, hamming = eval_multilabel_metrics(y_test, predicted_qualities)

    # Log das métricas
    mlflow.log_param("var_smoothing", var_smoothing)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("hamming_loss", hamming)

    # Registro do modelo
    mlflow.sklearn.log_model(model, "model")

    # Imprimir métricas
    print(f"Gaussian Naive Bayes model (var_smoothing={var_smoothing}):")
    print(f"  ACCURACY: {accuracy}")
    print(f"  HAMMING LOSS: {hamming}")


Gaussian Naive Bayes model (var_smoothing=2.451655277057877e-09):
  ACCURACY: 0.17635203224722876
  HAMMING LOSS: 0.17685589519650655


/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [19]:
file_path = '/content/mlruns/0/c7b95c54becd442cbe5f4a915a4adf6a/metrics/accuracy'

In [20]:
# Abrir o arquivo para leitura
with open(file_path, 'r') as file:
    content = file.read()

In [21]:
print(f"Acurácia do teste: {content}")

Acurácia do teste: 1706323310427 0.17635203224722876 0



#**7. Diagnóstico do Modelo**


Com base nas métricas obtidas - Acurácia (Accuracy): 0.1764 e Perda de Hamming: 0.1769 - embora esse modelo tenha se destacado como o melhor na métrica utilizada para otimização de hiperparâmetros, ele não demonstra um desempenho satisfatório na tarefa de análise de sentimentos em tweets. Os valores baixos tanto para a acurácia quanto para a perda de Hamming indicam que o modelo está enfrentando desafios significativos na predição dos rótulos de sentimento. Nesse contexto, é imperativo explorar outras abordagens de modelagem e considerar a implementação de técnicas adicionais de pré-processamento de dados para aprimorar o desempenho do modelo.